In [1]:
# import dill
import numpy as np
from matplotlib import figure, gridspec
# from pathlib import Path
# from astropy.table import Table
# from astropy import units as u
# from astropy.coordinates import SkyCoord

import scanner_interpretation as scani

import healpy as hp

from matplotlib import colors,cm
import matplotlib.pyplot as plt
from astropy import visualization as vis



bands = ["u_5_15", "u_15_25", "u_25_35", "u_35_45", "u_45_55", "u_55_65", "u_65_75", "u_75_90", "l_15_5", "l_25_15", "l_35_25", "l_45_35", "l_55_45", "l_65_55", "l_75_65", "l_90_75", "center"]
versions = ["hess1","hess1u"]
paths = [f"/lustre/fs22/group/hess/user/wybouwt/full_scanner_survey/{version}/{band}" for band in bands for version in versions]
# mplets = scani.Multiplets(paths[0])
# mplets.appendMultiplets(*[scani.Multiplets(path) for path in paths[1:]])
mplet_list = [scani.Multiplets(path) for path in paths]


unicorns = [9,18]
for j in unicorns:
    mplet_list[j].objectifyColumns()

    
mplets = mplet_list[0]
mplets.appendMultiplets(*mplet_list[1:])

Aitov scatter

In [ ]:
fig,ax = mplets.getAitoffFigure()
fig.savefig("figures/combined/aitoff.png",facecolor="white",dpi=300)

Cornerplot of the resulting multiplets

In [ ]:
# plotting mplet results
from corner import corner

fig = corner(mplets.table.to_pandas(),var_names=["Nmax","dt","da"])
fig.savefig("figures/combined/corner_center.png",facecolor="white",dpi=300)

HEALPix

In [ ]:
mask = mplets.table["Nmax"]>=3
cmap = cm.plasma
cmap.set_under("white")
m = scani.cat2hpx(mplets.table[mask]["MEDIAN_RA"],mplets.table[mask]["MEDIAN_DEC"],nside=2**7)
hp.mollview(m,cmap=cmap)
ax = plt.gca()
ax.images[0].set_norm(colors.PowerNorm(.2))
# plt.savefig("figures/combined/healpy.png",facecolor="white")

Simbad search

In [4]:
with open("output.txt","w") as f:
    f.write(mplets.constructSimbadCriteria(True))
# simbad_results = mplets.searchSimbad(coordinate_criteria=True)

In [5]:
print(simbad_results)

NameError: name 'simbad_results' is not defined

TeVCat search and distance histogram

In [ ]:
from tevcat import TeVCat
tevcat = TeVCat()
mplets.searchTeVCat(tevcat.sources)


vline_sourcebound = .2
hist = vis.hist(mplets.table["TEVCAT_DISTANCES_DEG"],bins="freedman",histtype="step")
ax = plt.gca()
ax.set_xlabel("Distance to nearest source [deg]")
ax.set_ylabel("Counts")
ax.vlines(vline_sourcebound,0,1e4,color="red",ls="--",label=f"{vline_sourcebound} deg")
ax.set_yscale("log")
ax.set_xlim(0,30)
ax.legend()
# ax.set_xscale("log")
plt.savefig("figures/combined/sourcedist_hist.png",facecolor="white")

Nmax histogram

In [ ]:
hist = vis.hist(mplets.table["Nmax"],bins=range(3,22),histtype="step")
ax = plt.gca()
ax.set_xlabel("Nmax")
ax.set_ylabel("Counts")
ax.set_yscale("log")
ax.set_xticks(range(3,22))
plt.savefig("figures/combined/Nmax_hist.png",facecolor="white")

In [ ]:
big_mplets = mplets.table["Nmax"] >= 7
mplets.table.group_by("SOURCE")
mplets.table[big_mplets]

In [ ]:
mplets.table["SOURCE"][1].getID()